In [31]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from itertools import product
from gensim.models import word2vec
import itertools
from tqdm import tqdm 

In [24]:
# Read recipe inputs
reviews_TF_IDF = dataiku.Dataset("reviews_TF_IDF")
df = reviews_TF_IDF.get_dataframe()

In [25]:
folder_path = dataiku.Folder("m9JZdV7b").get_path()
model_path = folder_path + "/word2vec_ramen_model.model"
ramen_model = word2vec.Word2Vec.load(model_path)

In [26]:
store_cross = []
for ids in product(df['id'], repeat=2):
    store_cross.append(ids)

In [27]:
store_cross_df = pd.DataFrame(store_cross, columns=['id_x', 'id_y'])

In [28]:
store_cross_detail = store_cross_df.merge(df[['id','store_name','score','review_cnt','texts_tfidf_sorted_top20']], 
                                          how='inner', 
                                          left_on='id_x', 
                                          right_on='id').drop(columns='id').merge(df[['id','store_name','score','review_cnt','texts_tfidf_sorted_top20']], 
                                                                                  how='inner', 
                                                                                  left_on='id_y', 
                                                                                  right_on='id').drop(columns='id')

In [29]:
store_cross_detail.head()

,id_x,id_y,store_name_x,score_x,review_cnt_x,texts_tfidf_sorted_top20_x,store_name_y,score_y,review_cnt_y,texts_tfidf_sorted_top20_y
0,ID-000001,ID-000001,手打式超多加水麺 ののくら,3.98,479,"['ワンタン', 'ストーブ', '予約', '2時間', '1時間', '記帳', '2時...",手打式超多加水麺 ののくら,3.98,479,"['ワンタン', 'ストーブ', '予約', '2時間', '1時間', '記帳', '2時..."
1,ID-000002,ID-000001,Homemade Ramen 麦苗,3.97,731,"['記帳', 'ボード', 'ウェイティング', '深谷', '9時', '記名', 'ワン...",手打式超多加水麺 ののくら,3.98,479,"['ワンタン', 'ストーブ', '予約', '2時間', '1時間', '記帳', '2時..."
2,ID-000003,ID-000001,麺尊 RAGE,3.96,723,"['金華', 'ハム', 'ロック', '穂先', '燻製', 'レモン', '豚バラ', ...",手打式超多加水麺 ののくら,3.98,479,"['ワンタン', 'ストーブ', '予約', '2時間', '1時間', '記帳', '2時..."
3,ID-000004,ID-000001,宍道湖しじみ中華蕎麦 琥珀,3.95,296,"['シジミ', 'バラ', '焼き', '紫玉ねぎ', '吊し', '菅野', 'ワンタン'...",手打式超多加水麺 ののくら,3.98,479,"['ワンタン', 'ストーブ', '予約', '2時間', '1時間', '記帳', '2時..."
4,ID-000005,ID-000001,らぁ麺や 嶋,3.95,177,"['記帳', '4点', '細切り', '記入', '炭火焼き', '昆布', '燻製', ...",手打式超多加水麺 ののくら,3.98,479,"['ワンタン', 'ストーブ', '予約', '2時間', '1時間', '記帳', '2時..."


In [30]:
# store_cross_detail = store_cross_detail[store_cross_detail['id_x'].isin(df['id'].loc[0:50])]
store_cross_detail = store_cross_detail.reset_index(drop=True).sort_values(['id_x'])

In [32]:
##ラーメン店xに対してラーメン店yの類似度を算出
#コサイン類似度を算出する関数を定義
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))

In [58]:
#cossimだけの組み合わせ（同じワード同士の組みあわせがでてくるため）
#2次元を１次元にする　setが重複を削除てきなやつ。
# uniq_words = list(set(itertools.chain.from_iterable(df['texts_tfidf_sorted_top20'].values)))
word_list = []
arr = df['texts_tfidf_sorted_top20'].values
for a in arr:
    l = a.replace("'", "").replace("[", "").replace("]", "").replace(" ", "").split(",")
    word_list.append(l)
uniq_words = list(set(itertools.chain.from_iterable(word_list)))    
scores = {}
for word1, word2 in product(uniq_words, repeat=2):
    # print(word1, word2)
    scores[(word1, word2)] =  cos_sim(ramen_model.wv[word1], ramen_model.wv[word2])

In [70]:
store_cross_detail['texts_tfidf_sorted_top20_x'][i].replace("'", "").replace("[", "").replace("]", "").replace(" ", "").split(",")

['ワンタン',
 'ストーブ',
 '予約',
 '2時間',
 '1時間',
 '記帳',
 '2時間半',
 'ブロス',
 '引き換え',
 '大葉',
 '出迎え',
 'モチ',
 'ローストポーク',
 '白髪',
 '鶏の胸肉',
 '文句なし',
 '三つ葉',
 '水分',
 '25人',
 'モッチモチ']

In [71]:
avg_avg_scores = []
for i in tqdm(range(len(store_cross_detail['texts_tfidf_sorted_top20_x']))):
    avg_scores = []
    x_list = store_cross_detail['texts_tfidf_sorted_top20_x'][i].replace("'", "").replace("[", "").replace("]", "").replace(" ", "").split(",")
    for j in range(len(x_list)):
        word_cross_scores = []
        word_a = x_list[j]
        for k in range(len(x_list)):
            word_b = x_list[k]
            score = scores[(word_a, word_b)]#単語間のスコアを出す。
            word_cross_scores.append(score)
        avg_scores.append(np.mean(word_cross_scores))#20個の単語間スコアの平均値
    avg_avg_scores.append(np.mean(avg_scores))#20個の単語間スコアの平均値の平均値
store_cross_detail.insert(6, 'avg_cos_sim_rate', avg_avg_scores)
# 「二郎」と類似度が高いラーメン屋を高い順に表示
store_cross_detail = store_cross_detail.sort_values(['id_x', 'avg_cos_sim_rate'], ascending=[True, False])
df_sim_x = store_cross_detail[store_cross_detail['store_name_x'].str.contains('二郎')]
df_sim_x.reset_index(drop=True)

def min_max(x, axis=None):
    min = x.min(axis=axis, keepdims=True)
    max = x.max(axis=axis, keepdims=True)
    result = (x-min)/(max-min)
    return result
b = df_sim_x['avg_cos_sim_rate']
c = min_max(b.values)
df_sim_x.insert(7, '正規化', c)
df_sim_x

100%|██████████| 44521/44521 [00:35<00:00, 1263.09it/s]


,id_x,id_y,store_name_x,score_x,review_cnt_x,texts_tfidf_sorted_top20_x,avg_cos_sim_rate,正規化,store_name_y,score_y,review_cnt_y,texts_tfidf_sorted_top20_y
20578,ID-000112,ID-000098,ラーメン二郎 ひばりヶ丘駅前店,3.76,422,"['整理券', 'ヒバジ', 'コール', '乳化', '野菜', 'クタ', '鍋二郎',...",0.364675,1.000000,麺処 銀笹,3.77,812,"['つみれ', '注ぎ口', '薬味', '水菜', 'ワタ', '糸唐辛子', 'レンコン..."
16780,ID-000112,ID-000080,ラーメン二郎 ひばりヶ丘駅前店,3.76,422,"['整理券', 'ヒバジ', 'コール', '乳化', '野菜', 'クタ', '鍋二郎',...",0.358617,0.964607,麺屋吉左右,3.79,925,"['女将', '夫婦', '奥様', '奥さん', 'ルール', '10人', 'お客様',..."
3698,ID-000112,ID-000018,ラーメン二郎 ひばりヶ丘駅前店,3.76,422,"['整理券', 'ヒバジ', 'コール', '乳化', '野菜', 'クタ', '鍋二郎',...",0.349240,0.909826,キング製麺,3.90,274,"['ワンタン', '山椒', '小松菜', '製麺', 'パンパン', 'かつお節', 'ナ..."
30284,ID-000112,ID-000144,ラーメン二郎 ひばりヶ丘駅前店,3.76,422,"['整理券', 'ヒバジ', 'コール', '乳化', '野菜', 'クタ', '鍋二郎',...",0.337528,0.841405,中華そば いづる,3.73,338,"['猪又', '濃密', '煮干し', '村上', '岩海苔', '朝日', '玉ねぎ', ..."
41467,ID-000112,ID-000197,ラーメン二郎 ひばりヶ丘駅前店,3.76,422,"['整理券', 'ヒバジ', 'コール', '乳化', '野菜', 'クタ', '鍋二郎',...",0.334009,0.820848,麺 酒 やまの,3.72,269,"['ニラ', 'トロ', 'ニンニク', '魚粉', 'キン', '卵黄', '挽肉', '..."
...,...,...,...,...,...,...,...,...,...,...,...,...
24254,ID-000201,ID-000115,ラーメン二郎 品川店,3.72,404,"['シナジ', '新代田', 'コール', 'ニンニク', '宣告', '乳化', 'キャベ...",0.204112,0.061987,中華そば ムタヒロ 1号店,3.76,466,"['岩海苔', '煮干し', '海藻', '岩のり', '麦わら帽子', '幅広', 'テー..."
33960,ID-000201,ID-000161,ラーメン二郎 品川店,3.72,404,"['シナジ', '新代田', 'コール', 'ニンニク', '宣告', '乳化', 'キャベ...",0.203171,0.056486,井田商店,3.73,237,"['強面', '真っ黒', 'トイレ', '緩やか', 'ドトール', '三河屋', '木造..."
13071,ID-000201,ID-000062,ラーメン二郎 品川店,3.72,404,"['シナジ', '新代田', 'コール', 'ニンニク', '宣告', '乳化', 'キャベ...",0.201258,0.045313,麺屋 はし本,3.81,291,"['ワンオペ', '辛味', '3枚', '豚骨', '三河屋', '格別', '手際', ..."
10539,ID-000201,ID-000050,ラーメン二郎 品川店,3.72,404,"['シナジ', '新代田', 'コール', 'ニンニク', '宣告', '乳化', 'キャベ...",0.197646,0.024208,鶏こく中華 すず喜,3.84,364,"['階段', '刺激的', '山椒', '三河屋', '刺激', 'ベンチ', '風合い',..."


In [0]:
# Compute recipe outputs from inputs
# TODO: Replace this part by your actual code that computes the output, as a Pandas dataframe
# NB: DSS also supports other kinds of APIs for reading and writing data. Please see doc.

similarity_scores_df = reviews_TF_IDF_df # For this sample code, simply copy input to output


# Write recipe outputs
similarity_scores = dataiku.Dataset("similarity_scores")
similarity_scores.write_with_schema(similarity_scores_df)